<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/%EB%8F%99%EC%A0%81%EC%9E%90%EC%82%B0%EB%B0%B0%EB%B6%84_%EA%B0%80%EC%86%8D_%EB%93%80%EC%96%BC%EB%AA%A8%EB%A9%98%ED%85%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
# !pip install yfinance
# !pip install pandas_market_calendars

In [66]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import csv

# 데이터 csv 내보내기 사용 여부
is_export_csv = True

# display에서 소수점 4자리까지 표현
pd.options.display.float_format = '{:.4f}'.format

# display시 DataFrame 모든 열 표시
pd.set_option('display.max_columns', None)

# 데이터 가져올 기간을 지정합니다. (오늘로부터 12개월 전)
today = datetime.today().date()
start_date = today - timedelta(days=375)

# 전략명
str_strategy = '가속 듀얼모멘텀 '

# 전략에 사용할 ETF 명 리스트에 담기
etf = ['SPY', 'SCZ', 'BIL', 'TLT']

# 자산군 리스트 더하기, 중복 제거
etf_order = etf

# ETF 가격 추출
df_etf = yf.download(etf_order, start=start_date, end=today)['Adj Close']\
.sort_index(ascending=False)[etf_order].rename_axis(None, axis=0)

display(df_etf)

[*********************100%%**********************]  4 of 4 completed


,SPY,SCZ,BIL,TLT
2023-10-19,426.4300,53.6400,91.6900,82.7700
2023-10-18,430.2100,53.9600,91.6500,84.5000
2023-10-17,436.0200,54.9600,91.6400,85.2500
2023-10-16,436.0400,54.9700,91.6300,86.2000
2023-10-13,431.5000,54.6300,91.6200,87.6100
...,...,...,...,...
2022-10-14,352.0873,47.3875,87.6071,95.5036
2022-10-13,360.2981,48.2823,87.6071,96.2980
2022-10-12,351.0339,47.2302,87.5784,97.2282
2022-10-11,352.1956,47.5055,87.5784,96.7534


In [67]:
# csv 경로 설정
output_path =  '/content/drive/MyDrive/DynamicAssetAllocation/'

# 오늘 날짜를 가져와서 파일명에 추가
today_date = datetime.now().strftime('%y%m%d')
filename = f'{today_date} {str_strategy}.xlsx'

# 오늘 날짜 string에 저장
str_today = today.strftime('### %Y-%m-%d ')

# 월평균영업일수. average Monthly Business Day
MBD = 21

# 인덱스 만들기
df_momentum_index = ['today/' + str(df_etf.index[0].date()), '1mo_ago/' + str(df_etf.index[MBD].date()), \
                     '3mo_ago/' + str(df_etf.index[MBD*3].date()), '6mo_ago/' + str(df_etf.index[MBD*6].date()), \
                     '12mo_ago/' + str(df_etf.index[MBD*12].date())]


## DataFrame 만들기: 1-3-6-12 개월 etf 가격
price_today = df_etf.iloc[0:1]
price_1mo = df_etf.shift(-MBD).iloc[0:1]
price_3mo = df_etf.shift(-MBD*3).iloc[0:1]
price_6mo = df_etf.shift(-MBD*6).iloc[0:1]
price_12mo = df_etf.shift(-MBD*12).iloc[0:1]

df_price = pd.concat([price_today, price_1mo, price_3mo, price_6mo, price_12mo])
df_price.index = df_momentum_index

str_etf_price = str_today + str_strategy + "### ETF 현재-1-3-6-12 개월 가격"
display(Markdown(str_etf_price))
display(df_price)

# csv에 제목 및 DataFrame 추가. 드라이브 마운트 후 해야함
if is_export_csv:
  with open(output_path + filename, 'w') as f:
    f.write(f'{str_etf_price}\n')
    df_price.to_csv(f, index=True)

### 2023-10-20 가속 듀얼모멘텀 ### ETF 현재-1-3-6-12 개월 가격

,SPY,SCZ,BIL,TLT
today/2023-10-19,426.4300,53.6400,91.6900,82.7700
1mo_ago/2023-09-20,438.6400,57.8600,91.2766,92.7961
3mo_ago/2023-07-21,450.5906,60.8900,90.5138,100.8277
6mo_ago/2023-04-20,408.9133,59.2368,89.4754,103.2421
12mo_ago/2022-10-18,365.3781,48.9805,87.6071,95.2613


In [68]:
## DataFrame 만들기: 수익률
ror_1mo = df_etf.pct_change(-MBD).iloc[0:1]*100    # 1개월 수익률
ror_3mo = df_etf.pct_change(-MBD*3).iloc[0:1]*100    # 3개월 수익률
ror_6mo = df_etf.pct_change(-MBD*6).iloc[0:1]*100   # 6개월 수익률
ror_12mo = df_etf.pct_change(-MBD*12).iloc[0:1]*100  # 12개월 수익률

df_etf_ror = pd.concat([ror_1mo, ror_3mo, ror_6mo, ror_12mo])
df_etf_ror.index = df_momentum_index[1:]

str_etf_ror = str_today + str_strategy + "### ETF 1-3-6-12 개월 수익률"
display(Markdown(str_etf_ror))
display(df_etf_ror)

# csv에 제목 및 DataFrame 추가
if is_export_csv:
  with open(output_path + filename, 'a') as f:
    f.write('\n')
    f.write(f'{str_etf_ror}\n')
    df_etf_ror.to_csv(f, index=True)

### 2023-10-20 가속 듀얼모멘텀 ### ETF 1-3-6-12 개월 수익률

,SPY,SCZ,BIL,TLT
1mo_ago/2023-09-20,-2.7836,-7.2935,0.4529,-10.8044
3mo_ago/2023-07-21,-5.3620,-11.9067,1.2995,-17.9094
6mo_ago/2023-04-20,4.2837,-9.4482,2.4751,-19.8292
12mo_ago/2022-10-18,16.7092,9.5129,4.6604,-13.1127


In [69]:
ror_average = (ror_1mo + ror_3mo + ror_6mo) / 3
ror_average.index = ['1-3-6mo_ror_average']
display(ror_average)

,SPY,SCZ,BIL,TLT
1-3-6mo_ror_average,-1.2873,-9.5495,1.4092,-16.1810


In [70]:
display(Markdown(str_today[1:] + str_strategy))

# SPY, SCZ 수익이 둘 다 BIL 보다 낮으면 TLT 에 투자
str_is_bond = 'False'
if (ror_average['SPY'].iloc[0] < ror_average['BIL'].iloc[0]) and (ror_average['SCZ'].iloc[0] < ror_average['BIL'].iloc[0]):
  str_is_bond = 'True'

str_etf = "## 미국 주식(SPY)과 글로벌 소형주(SCZ) 둘 다 초단기채(BIL) 보다 낮은가? " + str_is_bond
display(Markdown(str_etf))

str_higher_stock = ''

if str_is_bond == 'True':
  str_higher_stock = 'TLT'
else:
  # 아니라면 SPY, SCZ 중 12개월 수익률이 더 높은 것 표시
  if ror_average['SPY'].iloc[0] > ror_average['SCZ'].iloc[0]:
    str_higher_stock = 'SPY'
  else:
    str_higher_stock = 'SCZ'
str_higher_stock_text = "## 투자할 주식: " + str_higher_stock
display(Markdown(str_higher_stock_text))

# csv에 제목 및 DataFrame 추가
if is_export_csv:
  with open(output_path + filename, 'a') as f:
    f.write('\n')
    f.write(f'{str_today[1:] + str_strategy}\n')
    f.write(f'{str_etf}\n')
    f.write(f'{str_higher_stock_text}\n')

## 2023-10-20 가속 듀얼모멘텀 

## 미국 주식(SPY)과 글로벌 소형주(SCZ) 둘 다 초단기채(BIL) 보다 낮은가? True

## 투자할 주식: TLT